#**Imports and Util**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'AI Capstone/Colab Notebooks/Data Filtering'
assert FOLDERNAME is not None, "[!] Enter the foldername."


import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%load_ext autoreload
%autoreload 2

Mounted at /content/drive


In [2]:
#Was used for a test requirenemt. I might readd in the future, but right now it is not needed.
#!pip install sdv

In [3]:
import requests
import pandas as pd
from io import BytesIO

import torch.nn as nn
import torch as T
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

from collections import OrderedDict

from filtering_model import FilteringModel

In [4]:
#Credit to https://huggingface.co/thenlper/gte-base
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [5]:
def loadData(link):
  req = requests.get(link)
  content = BytesIO(req.content)
  df = pd.read_csv(content)
  return df

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np

#Function for testing a model
def test_model(predictions:pd.DataFrame,test_data:pd.DataFrame,name_column:str="ingredient",print_results=False):
  #Trim DataFrames
  test_data = test_data.drop(columns=[name_column])
  predictions = predictions.drop(columns=[name_column])

  #Convert to binary
  test_data = (test_data == 'yes').to_numpy()
  predictions = (predictions == 'yes').to_numpy()

  #Get Metrics
  accuracy = accuracy_score(test_data,predictions)
  precision = precision_score(test_data,predictions,average=None)
  recall = recall_score(test_data,predictions,average=None)

  #Print Results
  if print_results:
    print("Accuracy:",accuracy)
    print("Avg. Precision:",np.average(precision),"| Full Precision",precision)
    print("Avg. Recall",np.average(recall),"| Full Recall",recall)

  return [accuracy,precision,recall]

#Function for getting incorrect results of a model
def get_incorrect_results(predictions:pd.DataFrame,test_data:pd.DataFrame,name_column:str="ingredient"):
  #Reset indices of inputs for consistency
  test_data = test_data.reset_index(drop=True)
  predictions = predictions.reset_index(drop=True)

  #Gather false positives and negatives
  false_positives_df = (test_data == predictions) | (predictions == "no")
  false_negatives_df = (test_data == predictions) | (predictions == "yes")

  #Filter the data to show only rows with incorrect results
  incorrect_classifications = predictions[false_positives_df.eq(0).any(axis=1) | false_negatives_df.eq(0).any(axis=1)]
  incorrect_classifications = incorrect_classifications.where(false_positives_df==True,"false positive")
  incorrect_classifications = incorrect_classifications.where(false_negatives_df==True,"false negative")
  incorrect_classifications = incorrect_classifications.where((false_positives_df==False) | (false_negatives_df==False),"correct")

  #Reset the name column and return
  incorrect_classifications[name_column] = test_data[name_column]
  return incorrect_classifications

#**Dataset Loading and Analysis**

In [7]:
from sklearn.model_selection import train_test_split

food_data = loadData("https://raw.githubusercontent.com/ezramuskat/Ingredient-Substitution-Capstone/main/data_preparation/classification_dataset/common_ingredients_1000.csv")
food_data, food_test_data = train_test_split(food_data, test_size=0.1)

food_data

,ingredient,vegetarian,vegan,dairy_free,gluten_free
332,ground cardamom,yes,yes,yes,yes
757,whole cloves,yes,yes,yes,yes
319,refried beans,yes,yes,yes,yes
975,chickpea flour,yes,yes,yes,yes
265,hot pepper sauce,yes,yes,yes,yes
...,...,...,...,...,...
431,frozen corn,yes,yes,yes,yes
507,parsley sprigs,yes,yes,yes,yes
311,mint leaves,yes,yes,yes,yes
288,saffron threads,yes,yes,yes,yes


In [8]:
print((food_data[:] == 'yes').sum() / (food_data[:] != 'yes').sum())

ingredient      0.000000
vegetarian      6.250000
vegan           2.714876
dairy_free      8.879121
gluten_free    13.046875
dtype: float64


In [9]:
cheeseburger = ["salt","bread","meat","cheese","cheddar cheese","onions","chili","taco seasoned meat","worcestershire sauce","bug infested lettuce","cookie labeled eat me","lead pipe","your hopes and dreams"]

#**Filtering Model Class Experimentation**

##**facebook/drama-base**

In [ ]:
model_name = "facebook/drama-base"

###**Regression Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 14.197486400604248 | Val Loss: 3.458309829235077 | Train Acc: tensor(0.8195) | Val Acc: tensor(0.8889)
Epoch: 2 | Train Loss: 12.081097155809402 | Val Loss: 2.9427500665187836 | Train Acc: tensor(0.8529) | Val Acc: tensor(0.8889)
Epoch: 3 | Train Loss: 10.684749066829681 | Val Loss: 2.609096199274063 | Train Acc: tensor(0.8529) | Val Acc: tensor(0.8889)
Epoch: 4 | Train Loss: 9.849682420492172 | Val Loss: 2.4206523299217224 | Train Acc: tensor(0.8529) | Val Acc: tensor(0.8889)
Epoch: 5 | Train Loss: 9.297993153333664 | Val Loss: 2.210868716239929 | Train Acc: tensor(0.8529) | Val Acc: tensor(0.8889)
Epoch: 6 | Train Loss: 8.936812371015549 | Val Loss: 2.134146273136139 | Train Acc: tensor(0.8529) | Val Acc: tensor(0.8889)
Epoch: 7 | Train Loss: 8.690441220998764 | Val Loss: 2.0835153460502625 | Train Acc: tensor(0.8529) | Val Acc: tensor(0.8889)
Epoch: 8 | Train Loss: 8.464546412229538 | Val Loss: 2.045474797487259 | Train Acc: tensor(0.8529) | Val Acc: tensor(0.

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.81
Avg. Precision: 0.9470588235294117
Avg. Recall 0.9640718562874252


[0.81, 0.9470588235294117, 0.9640718562874252]

In [ ]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

modeling_drama.py:   0%|          | 0.00/6.08k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/facebook/drama-base:
- modeling_drama.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 13.168 | Val Loss: 3.67 | Train Acc: 0.259 | Val Acc: 0.658 | Train Pre: 0.975 | Val Pre: 0.971 | Train Rec: 0.975 | Val Rec: 0.971
Epoch: 2 | Train Loss: 10.311 | Val Loss: 2.576 | Train Acc: 0.501 | Val Acc: 0.623 | Train Pre: 0.985 | Val Pre: 0.968 | Train Rec: 0.985 | Val Rec: 0.968
Epoch: 3 | Train Loss: 8.105 | Val Loss: 2.089 | Train Acc: 0.708 | Val Acc: 0.717 | Train Pre: 0.993 | Val Pre: 0.977 | Train Rec: 0.993 | Val Rec: 0.977
Epoch: 4 | Train Loss: 6.295 | Val Loss: 1.556 | Train Acc: 0.824 | Val Acc: 0.846 | Train Pre: 0.989 | Val Pre: 0.96 | Train Rec: 0.989 | Val Rec: 0.96
Epoch: 5 | Train Loss: 4.64 | Val Loss: 1.346 | Train Acc: 0.911 | Val Acc: 0.869 | Train Pre: 0.989 | Val Pre: 0.968 | Train Rec: 0.989 | Val Rec: 0.968
Epoch: 6 | Train Loss: 3.312 | Val Loss: 1.288 | Train Acc: 0.939 | Val Acc: 0.853 | Train Pre: 0.991 | Val Pre: 0.967 | Train Rec: 0.991 | Val Rec: 0.967
Epoch: 7 | Train Loss: 2.828 | Val Loss: 1.073 | Train Acc: 0.942 | Val 

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.89
Avg. Precision: 0.9821807425540596 | Full Precision [0.98837209 0.97368421 1.         0.96666667]
Avg. Recall 0.9759391771019679 | Full Recall [0.98837209 0.94871795 1.         0.96666667]


[0.89,
 array([0.98837209, 0.97368421, 1.        , 0.96666667]),
 array([0.98837209, 0.94871795, 1.        , 0.96666667])]

In [ ]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 12.727 | Val Loss: 3.808 | Train Acc: 0.344 | Val Acc: 0.666 | Train Pre: 0.987 | Val Pre: 0.969 | Train Rec: 0.987 | Val Rec: 0.969
Epoch: 2 | Train Loss: 10.031 | Val Loss: 2.693 | Train Acc: 0.548 | Val Acc: 0.728 | Train Pre: 0.994 | Val Pre: 0.979 | Train Rec: 0.994 | Val Rec: 0.979
Epoch: 3 | Train Loss: 8.628 | Val Loss: 2.414 | Train Acc: 0.669 | Val Acc: 0.732 | Train Pre: 0.993 | Val Pre: 0.965 | Train Rec: 0.993 | Val Rec: 0.965
Epoch: 4 | Train Loss: 7.374 | Val Loss: 2.108 | Train Acc: 0.757 | Val Acc: 0.741 | Train Pre: 0.995 | Val Pre: 0.961 | Train Rec: 0.995 | Val Rec: 0.961
Epoch: 5 | Train Loss: 6.145 | Val Loss: 2.183 | Train Acc: 0.832 | Val Acc: 0.751 | Train Pre: 0.996 | Val Pre: 0.967 | Train Rec: 0.996 | Val Rec: 0.967
Epoch: 6 | Train Loss: 4.955 | Val Loss: 1.685 | Train Acc: 0.868 | Val Acc: 0.804 | Train Pre: 0.997 | Val Pre: 0.963 | Train Rec: 0.997 | Val Rec: 0.963
Epoch: 7 | Train Loss: 4.096 | Val Loss: 1.498 | Train Acc: 0.905 | 

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.87
Avg. Precision: 0.9787567084078712 | Full Precision [0.97674419 0.96153846 1.         0.97674419]
Avg. Recall 0.9679039952295767 | Full Recall [0.97674419 0.96153846 1.         0.93333333]


[0.87,
 array([0.97674419, 0.96153846, 1.        , 0.97674419]),
 array([0.97674419, 0.96153846, 1.        , 0.93333333])]

In [ ]:
del fmodel_internal_model
del fmodel

##**sentence-transformers/all-MiniLM-L6-v2**

In [10]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

###**Regression Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(384, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 14.540719747543335 | Val Loss: 3.8001736998558044 | Train Acc: tensor(0.8053) | Val Acc: tensor(0.8375)
Epoch: 2 | Train Loss: 13.34915167093277 | Val Loss: 3.5520265102386475 | Train Acc: tensor(0.8620) | Val Acc: tensor(0.8375)
Epoch: 3 | Train Loss: 12.367191016674042 | Val Loss: 3.3069397807121277 | Train Acc: tensor(0.8620) | Val Acc: tensor(0.8375)
Epoch: 4 | Train Loss: 11.531408071517944 | Val Loss: 3.1223100423812866 | Train Acc: tensor(0.8620) | Val Acc: tensor(0.8375)
Epoch: 5 | Train Loss: 10.861199349164963 | Val Loss: 2.9718908965587616 | Train Acc: tensor(0.8620) | Val Acc: tensor(0.8375)
Epoch: 6 | Train Loss: 10.279071539640427 | Val Loss: 2.871199607849121 | Train Acc: tensor(0.8620) | Val Acc: tensor(0.8375)
Epoch: 7 | Train Loss: 9.829226583242416 | Val Loss: 2.7469168603420258 | Train Acc: tensor(0.8620) | Val Acc: tensor(0.8375)
Epoch: 8 | Train Loss: 9.408913791179657 | Val Loss: 2.6560305655002594 | Train Acc: tensor(0.8620) | Val Acc: ten

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.8
Avg. Precision: 0.9598853868194842
Avg. Recall 0.9710144927536232


[0.8, 0.9598853868194842, 0.9710144927536232]

In [ ]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [16]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(384, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [17]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 13.901 | Val Loss: 3.656 | Train Acc: 0.235 | Val Acc: 0.523 | Train Pre: 0.981 | Val Pre: 0.988 | Train Rec: 0.981 | Val Rec: 0.988
Epoch: 2 | Train Loss: 10.907 | Val Loss: 2.594 | Train Acc: 0.437 | Val Acc: 0.669 | Train Pre: 0.994 | Val Pre: 0.984 | Train Rec: 0.994 | Val Rec: 0.984
Epoch: 3 | Train Loss: 8.711 | Val Loss: 2.154 | Train Acc: 0.642 | Val Acc: 0.717 | Train Pre: 0.994 | Val Pre: 0.986 | Train Rec: 0.994 | Val Rec: 0.986
Epoch: 4 | Train Loss: 6.98 | Val Loss: 1.786 | Train Acc: 0.772 | Val Acc: 0.797 | Train Pre: 0.993 | Val Pre: 0.977 | Train Rec: 0.993 | Val Rec: 0.977
Epoch: 5 | Train Loss: 5.411 | Val Loss: 1.527 | Train Acc: 0.872 | Val Acc: 0.847 | Train Pre: 0.996 | Val Pre: 0.974 | Train Rec: 0.996 | Val Rec: 0.974
Epoch: 6 | Train Loss: 4.127 | Val Loss: 1.185 | Train Acc: 0.92 | Val Acc: 0.864 | Train Pre: 0.992 | Val Pre: 0.966 | Train Rec: 0.992 | Val Rec: 0.966
Epoch: 7 | Train Loss: 2.907 | Val Loss: 1.014 | Train Acc: 0.944 | Va

In [18]:
#Test the model
test_results = fmodel.filter(food_data['ingredient'].to_list())
test_model(test_results,food_data,print_results=True)
get_incorrect_results(test_results,food_data)

Accuracy: 0.9755283648498332
Avg. Precision: 0.9946407063821954 | Full Precision [0.99613402 0.99086758 0.99752781 0.99403341]
Avg. Recall 0.9961635253510427 | Full Recall [0.99741935 0.99086758 0.99876238 0.99760479]


,ingredient,vegetarian,vegan,dairy_free,gluten_free
55,red wine vinegar,correct,false positive,correct,correct
86,couscous,correct,false negative,correct,false positive
150,stock,correct,false positive,correct,correct
194,condensed cream of chicken soup,correct,correct,correct,false negative
211,dark soy sauce,correct,correct,correct,false positive
220,crabmeat,false positive,false positive,correct,correct
379,heavy whipping cream,correct,false negative,correct,correct
398,clam juice,correct,false negative,correct,correct
439,pepperoni,false positive,false positive,correct,correct
451,spaghetti,correct,correct,correct,false positive


In [14]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(384, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 12.62 | Val Loss: 3.366 | Train Acc: 0.378 | Val Acc: 0.523 | Train Pre: 0.988 | Val Pre: 0.974 | Train Rec: 0.988 | Val Rec: 0.974
Epoch: 2 | Train Loss: 9.938 | Val Loss: 2.592 | Train Acc: 0.57 | Val Acc: 0.759 | Train Pre: 0.992 | Val Pre: 0.989 | Train Rec: 0.992 | Val Rec: 0.989
Epoch: 3 | Train Loss: 8.418 | Val Loss: 2.162 | Train Acc: 0.693 | Val Acc: 0.807 | Train Pre: 0.996 | Val Pre: 0.986 | Train Rec: 0.996 | Val Rec: 0.986
Epoch: 4 | Train Loss: 7.545 | Val Loss: 2.044 | Train Acc: 0.734 | Val Acc: 0.827 | Train Pre: 0.994 | Val Pre: 0.983 | Train Rec: 0.994 | Val Rec: 0.983
Epoch: 5 | Train Loss: 6.237 | Val Loss: 1.747 | Train Acc: 0.811 | Val Acc: 0.858 | Train Pre: 0.995 | Val Pre: 0.976 | Train Rec: 0.995 | Val Rec: 0.976
Epoch: 6 | Train Loss: 5.21 | Val Loss: 1.693 | Train Acc: 0.876 | Val Acc: 0.841 | Train Pre: 0.994 | Val Pre: 0.98 | Train Rec: 0.994 | Val Rec: 0.98
Epoch: 7 | Train Loss: 4.259 | Val Loss: 1.479 | Train Acc: 0.906 | Val Ac

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.82
Avg. Precision: 0.950372041830327 | Full Precision [0.9625     0.92424242 0.96629213 0.94845361]
Avg. Recall 0.9564627395902181 | Full Recall [0.9625     0.89705882 0.96629213 1.        ]


[0.82,
 array([0.9625    , 0.92424242, 0.96629213, 0.94845361]),
 array([0.9625    , 0.89705882, 0.96629213, 1.        ])]

In [ ]:
del fmodel_internal_model
del fmodel

##**nomic-ai/nomic-embed-text-v1**

In [ ]:
model_name = "nomic-ai/nomic-embed-text-v1"

###**Regression Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

In [ ]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

In [ ]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

In [ ]:
del fmodel_internal_model
del fmodel

##**jonny9f/food_embeddings2**

In [ ]:
model_name = "jonny9f/food_embeddings2"

###**Regression Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 14.909412205219269 | Val Loss: 3.92104035615921 | Train Acc: tensor(0.7152) | Val Acc: tensor(0.8014)
Epoch: 2 | Train Loss: 14.118975043296814 | Val Loss: 3.709840714931488 | Train Acc: tensor(0.8015) | Val Acc: tensor(0.8111)
Epoch: 3 | Train Loss: 13.390045583248138 | Val Loss: 3.5207648873329163 | Train Acc: tensor(0.8154) | Val Acc: tensor(0.8222)
Epoch: 4 | Train Loss: 12.786079347133636 | Val Loss: 3.335104286670685 | Train Acc: tensor(0.8265) | Val Acc: tensor(0.8236)
Epoch: 5 | Train Loss: 12.247783362865448 | Val Loss: 3.2037811279296875 | Train Acc: tensor(0.8324) | Val Acc: tensor(0.8333)
Epoch: 6 | Train Loss: 11.774306386709213 | Val Loss: 3.086324542760849 | Train Acc: tensor(0.8383) | Val Acc: tensor(0.8444)
Epoch: 7 | Train Loss: 11.338542699813843 | Val Loss: 2.991334408521652 | Train Acc: tensor(0.8432) | Val Acc: tensor(0.8472)
Epoch: 8 | Train Loss: 10.971035093069077 | Val Loss: 2.8594135642051697 | Train Acc: tensor(0.8508) | Val Acc: tenso

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.67
Avg. Precision: 0.9260355029585798
Avg. Recall 0.8994252873563219


[0.67, 0.9260355029585798, 0.8994252873563219]

In [ ]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 12.95845514535904 | Val Loss: 3.535942554473877 | Train Acc: tensor(0.7111) | Val Acc: tensor(0.8819)
Epoch: 2 | Train Loss: 10.421903043985367 | Val Loss: 2.485925018787384 | Train Acc: tensor(0.8289) | Val Acc: tensor(0.8931)
Epoch: 3 | Train Loss: 8.90978991985321 | Val Loss: 2.27858966588974 | Train Acc: tensor(0.8912) | Val Acc: tensor(0.9181)
Epoch: 4 | Train Loss: 7.501890540122986 | Val Loss: 1.8162135183811188 | Train Acc: tensor(0.9249) | Val Acc: tensor(0.9333)
Epoch: 5 | Train Loss: 6.087361007928848 | Val Loss: 1.6769733130931854 | Train Acc: tensor(0.9444) | Val Acc: tensor(0.9458)
Epoch: 6 | Train Loss: 4.768336787819862 | Val Loss: 1.4113554656505585 | Train Acc: tensor(0.9652) | Val Acc: tensor(0.9486)
Epoch: 7 | Train Loss: 3.850701928138733 | Val Loss: 1.2969915717840195 | Train Acc: tensor(0.9684) | Val Acc: tensor(0.9472)
Epoch: 8 | Train Loss: 3.2543856352567673 | Val Loss: 1.2431402876973152 | Train Acc: tensor(0.9691) | Val Acc: tensor(0.9

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.83
Avg. Precision: 0.9464788732394366
Avg. Recall 0.9655172413793104


[0.83, 0.9464788732394366, 0.9655172413793104]

In [ ]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 13.18944177031517 | Val Loss: 3.503300130367279 | Train Acc: tensor(0.7142) | Val Acc: tensor(0.9167)
Epoch: 2 | Train Loss: 10.86081850528717 | Val Loss: 2.866569608449936 | Train Acc: tensor(0.8202) | Val Acc: tensor(0.8639)
Epoch: 3 | Train Loss: 9.51228678226471 | Val Loss: 2.504385083913803 | Train Acc: tensor(0.8748) | Val Acc: tensor(0.8889)
Epoch: 4 | Train Loss: 8.263584673404694 | Val Loss: 2.3024676740169525 | Train Acc: tensor(0.9172) | Val Acc: tensor(0.9153)
Epoch: 5 | Train Loss: 7.0020109713077545 | Val Loss: 2.062433809041977 | Train Acc: tensor(0.9374) | Val Acc: tensor(0.9097)
Epoch: 6 | Train Loss: 6.148843824863434 | Val Loss: 2.003048211336136 | Train Acc: tensor(0.9430) | Val Acc: tensor(0.9097)
Epoch: 7 | Train Loss: 5.600628167390823 | Val Loss: 1.6800348460674286 | Train Acc: tensor(0.9430) | Val Acc: tensor(0.9222)
Epoch: 8 | Train Loss: 4.545795664191246 | Val Loss: 1.5131531655788422 | Train Acc: tensor(0.9649) | Val Acc: tensor(0.940

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.81
Avg. Precision: 0.9512893982808023
Avg. Recall 0.9540229885057471


[0.81, 0.9512893982808023, 0.9540229885057471]

In [ ]:
del fmodel_internal_model
del fmodel

##**DivyaMereddy007/RecipeBert_v5**

In [ ]:
model_name = "DivyaMereddy007/RecipeBert_v5"

###**Regression Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 14.303769826889038 | Val Loss: 3.5683672428131104 | Train Acc: tensor(0.8015) | Val Acc: tensor(0.8750)
Epoch: 2 | Train Loss: 12.322549939155579 | Val Loss: 3.081792265176773 | Train Acc: tensor(0.8561) | Val Acc: tensor(0.8750)
Epoch: 3 | Train Loss: 10.899504661560059 | Val Loss: 2.758462578058243 | Train Acc: tensor(0.8561) | Val Acc: tensor(0.8750)
Epoch: 4 | Train Loss: 9.913725018501282 | Val Loss: 2.5731509625911713 | Train Acc: tensor(0.8564) | Val Acc: tensor(0.8750)
Epoch: 5 | Train Loss: 9.21401858329773 | Val Loss: 2.348560184240341 | Train Acc: tensor(0.8574) | Val Acc: tensor(0.8764)
Epoch: 6 | Train Loss: 8.716430366039276 | Val Loss: 2.267237573862076 | Train Acc: tensor(0.8599) | Val Acc: tensor(0.8764)
Epoch: 7 | Train Loss: 8.291297882795334 | Val Loss: 2.2038800716400146 | Train Acc: tensor(0.8630) | Val Acc: tensor(0.8792)
Epoch: 8 | Train Loss: 7.99583637714386 | Val Loss: 2.12089142203331 | Train Acc: tensor(0.8640) | Val Acc: tensor(0.880

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.75
Avg. Precision: 0.9228571428571428
Avg. Recall 0.9641791044776119


[0.75, 0.9228571428571428, 0.9641791044776119]

In [ ]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 13.061242699623108 | Val Loss: 3.4863703846931458 | Train Acc: tensor(0.7058) | Val Acc: tensor(0.9097)
Epoch: 2 | Train Loss: 10.276096135377884 | Val Loss: 2.619672328233719 | Train Acc: tensor(0.8328) | Val Acc: tensor(0.8986)
Epoch: 3 | Train Loss: 8.65947312116623 | Val Loss: 2.3732148110866547 | Train Acc: tensor(0.8905) | Val Acc: tensor(0.9194)
Epoch: 4 | Train Loss: 6.770247012376785 | Val Loss: 1.801015466451645 | Train Acc: tensor(0.9329) | Val Acc: tensor(0.9472)
Epoch: 5 | Train Loss: 5.478828802704811 | Val Loss: 1.5887492150068283 | Train Acc: tensor(0.9621) | Val Acc: tensor(0.9458)
Epoch: 6 | Train Loss: 4.006493538618088 | Val Loss: 1.3185678720474243 | Train Acc: tensor(0.9684) | Val Acc: tensor(0.9528)
Epoch: 7 | Train Loss: 3.331235535442829 | Val Loss: 1.1360452473163605 | Train Acc: tensor(0.9750) | Val Acc: tensor(0.9500)
Epoch: 8 | Train Loss: 2.5800505876541138 | Val Loss: 1.1287365183234215 | Train Acc: tensor(0.9795) | Val Acc: tensor(

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.78
Avg. Precision: 0.9413489736070382
Avg. Recall 0.9582089552238806


[0.78, 0.9413489736070382, 0.9582089552238806]

In [ ]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [ ]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [ ]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 1 | Train Loss: 13.036907434463501 | Val Loss: 4.001888871192932 | Train Acc: tensor(0.7121) | Val Acc: tensor(0.6222)
Epoch: 2 | Train Loss: 10.781681597232819 | Val Loss: 2.8004705905914307 | Train Acc: tensor(0.8098) | Val Acc: tensor(0.8694)
Epoch: 3 | Train Loss: 9.55362132191658 | Val Loss: 2.5351900458335876 | Train Acc: tensor(0.8651) | Val Acc: tensor(0.9125)
Epoch: 4 | Train Loss: 8.13015303015709 | Val Loss: 2.215449780225754 | Train Acc: tensor(0.9079) | Val Acc: tensor(0.9056)
Epoch: 5 | Train Loss: 6.96099366247654 | Val Loss: 1.9628736972808838 | Train Acc: tensor(0.9301) | Val Acc: tensor(0.9292)
Epoch: 6 | Train Loss: 5.849576354026794 | Val Loss: 1.7131022661924362 | Train Acc: tensor(0.9471) | Val Acc: tensor(0.9431)
Epoch: 7 | Train Loss: 5.279292866587639 | Val Loss: 1.7402592599391937 | Train Acc: tensor(0.9586) | Val Acc: tensor(0.9444)
Epoch: 8 | Train Loss: 4.313445717096329 | Val Loss: 1.3577772974967957 | Train Acc: tensor(0.9708) | Val Acc: tensor(0.9

In [ ]:
#Test the model
test_results = fmodel.filter(food_test_data['ingredient'].to_list())
test_model(test_results,food_test_data,print_results=True)
get_incorrect_results(test_results,food_test_data)

Accuracy: 0.78
Avg. Precision: 0.9204545454545454
Avg. Recall 0.9671641791044776


[0.78, 0.9204545454545454, 0.9671641791044776]

In [ ]:
del fmodel_internal_model
del fmodel